In [ ]:
pip install bertopic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.7/143.7 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.8/88.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.

In [ ]:
import pandas as pd
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA
from umap import UMAP
from sentence_transformers import SentenceTransformer
import nltk
from nltk.corpus import stopwords

In [ ]:
df = pd.read_csv("/content/cases.csv")
df.head()

,id,type,title,url,description,featured,post_date,updated_date,creator_id,creator_name,...,formal_evaluation,body,files_count,links_count,photos_count,videos_count,audio_count,evaluation_reports_count,evaluation_links_count,collections
0,1,case,British Columbia Citizens' Assembly on Elector...,https://participedia.net/case/1,"An independent, non-partisan assembly of 160 r...",0,2009-09-16,2021-04-12,4,Arfung,...,1.0,Problems and Purpose The Citizens' Assembly...,762,53,312,216,2,2,2,"(6786,collection,""OECD Project on Representati..."
1,2,case,Minneapolis Neighborhood Revitalization Program,https://participedia.net/case/2,The Minneapolis Neighborhood Revitalization Pr...,0,2009-09-17,2020-08-17,14,Annayork,...,NaN,Problems and Purpose The Minneapolis Neighbo...,2,1220,191,2,2,2,2,NaN
2,3,case,Wenling City Deliberative Poll,https://participedia.net/case/3,One of China’s most innovative forms of reinve...,0,2009-10-23,2020-08-04,24,Swdgray,...,1.0,"Problems and Purpose In late 2004, local gov...",2,217,235,2,2,2,274,"(6786,collection,""OECD Project on Representati..."
3,4,case,Citizens' Forum Europe,https://participedia.net/case/4,Citizens' Forum Europe was designed to allow d...,0,2009-11-04,2020-08-27,1027,"""Patrizia Nanz""",...,NaN,Problems and Purpose Fuelled by EU skepticis...,2,305,121,2,2,2,2,NaN
4,5,case,Biobanking in British Columbia,https://participedia.net/case/5,Two deliberative forums involving members of t...,0,2009-11-26,2019-11-27,32,Kcodoher,...,1.0,Problems and Purpose Biobanks have been hail...,2,2,120,2,2,2,497,NaN


In [ ]:
columns = ["title", "description", "body"]
df = df[columns]
df.head()

,title,description,body
0,British Columbia Citizens' Assembly on Elector...,"An independent, non-partisan assembly of 160 r...",Problems and Purpose The Citizens' Assembly...
1,Minneapolis Neighborhood Revitalization Program,The Minneapolis Neighborhood Revitalization Pr...,Problems and Purpose The Minneapolis Neighbo...
2,Wenling City Deliberative Poll,One of China’s most innovative forms of reinve...,"Problems and Purpose In late 2004, local gov..."
3,Citizens' Forum Europe,Citizens' Forum Europe was designed to allow d...,Problems and Purpose Fuelled by EU skepticis...
4,Biobanking in British Columbia,Two deliberative forums involving members of t...,Problems and Purpose Biobanks have been hail...


In [ ]:
# Combine text fields for better topic modeling
df["text"] = df["description"].fillna("") + " " + df["body"].fillna("")+" "+df["title"].fillna("")

In [ ]:
# Initialize a sentence transformer model for embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Convert text to embeddings
embeddings = embedding_model.encode(df["text"].tolist(), show_progress_bar=True)

Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
# Initialize UMAP for dimensionality reduction
umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.1, metric='cosine')

In [ ]:
# Download NLTK stop words
nltk.download("stopwords")
custom_stop_words = stopwords.words("english")

# Initialize vectorizer with stop words
vectorizer_model = CountVectorizer(stop_words=custom_stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# Create BERTopic model
topic_model = BERTopic(embedding_model=embedding_model, umap_model=umap_model, vectorizer_model=vectorizer_model)

In [ ]:
# Fit model to text data
topics, probs = topic_model.fit_transform(df["text"].tolist(), embeddings)

In [ ]:
# Add extracted topic info to the dataframe
df["topic"] = topics
df["topic_name"] = df["topic"].apply(lambda x: topic_model.get_topic(x) if x != -1 else "No Topic")

In [ ]:
# Extract keywords for each topic
topic_keywords = topic_model.get_topics()

In [ ]:
# Display top topics
print(topic_model.get_topic_info())

    Topic  Count                                              Name  \
0      -1    903         -1_process_community_public_participation   
1       0    122                  0_climate_assembly_change_energy   
2       1     93            1_budgeting_participatory_community_pb   
3       2     93                         2_community_jury_au_panel   
4       3     76                  3_quot_project_nbsp_municipality   
5       4     71                       4_vauban_city_planning_oecd   
6       5     58                5_research_dialogue_public_science   
7       6     56                          6_covid_19_pandemic_fund   
8       7     56                       7_health_care_mental_canada   
9       8     45            8_corruption_government_land_political   
10      9     40                9_internet_online_digital_citizens   
11     10     37                    10_european_eu_europe_citizens   
12     11     36              11_water_public_watershed_management   
13     12     34    

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Load a pre-trained T5 model and tokenizer
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [ ]:
# Function to generate a label using T5
def generate_label_with_t5(keywords):
    input_text = "generate a topic label: " + ", ".join(keywords)
    inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(inputs, max_length=20, num_beams=4, early_stopping=True)
    label = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return label

In [ ]:
# Generate topic labels using T5
topic_labels_t5 = {}
for topic_id in topic_model.get_topic_info()["Topic"]:
    if topic_id == -1:  # Skip outliers
        topic_labels_t5[topic_id] = "No Topic"
    else:
        # Get top keywords for the topic
        top_keywords = [word for word, _ in topic_model.get_topic(topic_id)][:5]
        # Generate a label using T5
        topic_labels_t5[topic_id] = generate_label_with_t5(top_keywords)

In [ ]:
# Add T5-generated topic labels to the dataframe
df["topic_label_t5"] = df["topic"].map(topic_labels_t5)

# Display the dataframe with T5-generated topic labels
print(df[["text", "topic", "topic_label_t5"]])

                                                   text  topic  \
0     An independent, non-partisan assembly of 160 r...     -1   
1     The Minneapolis Neighborhood Revitalization Pr...     -1   
2     One of China’s most innovative forms of reinve...     20   
3     Citizens' Forum Europe was designed to allow d...     10   
4     Two deliberative forums involving members of t...      5   
...                                                 ...    ...   
2267  Beyoğlu Heritage Days is an event aimed at pre...     33   
2268  Bologna is the first city in Italy that, after...      0   
2269  #ShareKoLang was a one-day deliberative dialog...      9   
2270                                                        -1   
2271  In 2005, Taiwan's Smangus community faced a le...     -1   

                                         topic_label_t5  
0                                              No Topic  
1                                              No Topic  
2     a topic label: china, budge